## Filtering SAM 

This job to filter SAM files output from Nextgenmap. It runs `samtools` to convert to BAM and store only human unmapped content.

In [ ]:
import pysam
import os
from hops import hdfs
import utils
import sys
from pyspark import SparkContext


In [ ]:
args=utils.load_arguments(sys.argv)
if args is not None:
    args=args['FilterSAM']
else :
    sys.exit(utils.NO_CONFIG_ERR)
    
sc = SparkContext.getOrCreate()

In [ ]:

inputRoot=args['INPUT_ROOT']
outputBam=args['OUTPUT_BAM']
outputMerge=args['OUTPUT_MERGE']
outputSort=args['OUTPUT_SORT']
outputFastq=args['OUTPUT_FASTQ']
threads=str(args['THREADS'])

In [ ]:

def convert_sam(file):
    
    print("INFO: Run unmapped sequences BAM : ", file)    
    bam_file=file.split('.')[0]+'.bam'
    hdfs.copy_to_local(os.path.join(inputRoot,file), overwrite=True)
    pysam.view('-o', bam_file, '-b', file,'-f 4','-@',threads, catch_stdout=False)
    if os.path.exists(bam_file):
        hdfs.copy_to_hdfs(bam_file,outputBam,overwrite=True)
        os.remove(bam_file)
        os.remove(file)

    return bam_file




    


In [ ]:

inputFiles=utils.load_file_names(inputRoot)

### convert to bam and filter unmapped sequences
unMapped=sc.parallelize(inputFiles).map(convert_sam).collect()
